# OTEX Uncertainty Analysis

This notebook demonstrates comprehensive uncertainty and sensitivity analysis for OTEC techno-economic assessments.

## Contents
1. [Setup](#1.-Setup)
2. [Monte Carlo Analysis](#2.-Monte-Carlo-Analysis)
3. [Tornado Diagram](#3.-Tornado-Diagram)
4. [Sobol Sensitivity](#4.-Sobol-Sensitivity-Analysis)
5. [Custom Parameters](#5.-Custom-Parameters)
6. [Comparative Analysis](#6.-Comparative-Analysis)

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# OTEX uncertainty analysis imports
from otex.analysis import (
    MonteCarloAnalysis,
    UncertaintyConfig,
    UncertainParameter,
    TornadoAnalysis,
    get_default_parameters,
    plot_histogram,
    plot_tornado,
    plot_cumulative_distribution,
    plot_scatter_matrix,
    create_summary_figure
)

# Plot settings
plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

print("OTEX Uncertainty Analysis Module loaded successfully!")

## 1. Setup

Define the base case conditions for analysis.

In [ ]:
# Base case: Jamaica-like conditions
T_WW = 28.0  # Warm water temperature (°C)
T_CW = 5.0   # Cold water temperature (°C)
P_GROSS = -50000  # 50 MW plant
COST_LEVEL = 'low_cost'

print(f"Base Case Configuration")
print(f"=" * 40)
print(f"Warm water temperature: {T_WW} °C")
print(f"Cold water temperature: {T_CW} °C")
print(f"Temperature difference: {T_WW - T_CW} °C")
print(f"Plant size: {-P_GROSS/1000} MW")
print(f"Cost level: {COST_LEVEL}")

In [ ]:
# View default uncertain parameters
params = get_default_parameters()

print(f"Default Uncertain Parameters ({len(params)} total)")
print(f"=" * 70)
print(f"{'Parameter':<40} {'Nominal':>10} {'Distribution':>12} {'Bounds'}")
print(f"-" * 70)
for p in params:
    print(f"{p.name:<40} {p.nominal:>10.3f} {p.distribution:>12} {p.bounds}")

## 2. Monte Carlo Analysis

Propagate uncertainty through the model using Latin Hypercube Sampling.

In [ ]:
# Configure Monte Carlo analysis
config = UncertaintyConfig(
    n_samples=500,    # Number of samples (increase for production)
    seed=42,          # Random seed for reproducibility
    parallel=True     # Use parallel processing
)

print(f"Monte Carlo Configuration")
print(f"Samples: {config.n_samples}")
print(f"Parameters: {config.n_params}")
print(f"Parallel: {config.parallel}")

In [ ]:
# Run Monte Carlo analysis
mc = MonteCarloAnalysis(
    T_WW=T_WW,
    T_CW=T_CW,
    config=config,
    p_gross=P_GROSS,
    cost_level=COST_LEVEL
)

print("Running Monte Carlo simulation...")
mc_results = mc.run(show_progress=True)
print("Done!")

In [ ]:
# Compute and display statistics
stats = mc_results.compute_statistics()

print(f"\nMonte Carlo Results ({config.n_samples} samples)")
print(f"=" * 50)

for output in ['lcoe', 'net_power', 'capex']:
    s = stats[output]
    print(f"\n{output.upper()}")
    print(f"  Mean:     {s[f'{output}_mean']:,.2f}")
    print(f"  Std Dev:  {s[f'{output}_std']:,.2f}")
    print(f"  CV:       {s[f'{output}_cv']:.1%}")
    print(f"  Median:   {s[f'{output}_median']:,.2f}")
    print(f"  P5-P95:   [{s[f'{output}_p5']:,.2f}, {s[f'{output}_p95']:,.2f}]")

In [ ]:
# Visualize LCOE distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
plot_histogram(mc_results, output='lcoe', ax=axes[0], bins=40)
axes[0].set_title('LCOE Distribution (Histogram)', fontsize=12, fontweight='bold')

# Cumulative distribution
plot_cumulative_distribution(mc_results, output='lcoe', ax=axes[1])
axes[1].set_title('LCOE Cumulative Distribution', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig('mc_lcoe_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Confidence intervals
print("LCOE Confidence Intervals")
print("=" * 40)

for conf in [0.50, 0.80, 0.90, 0.95]:
    low, high = mc_results.get_confidence_interval('lcoe', confidence=conf)
    print(f"{conf:.0%} CI: [{low:.2f}, {high:.2f}] ct/kWh")

## 3. Tornado Diagram

Identify which parameters have the largest impact on LCOE.

In [ ]:
# Run tornado analysis
tornado = TornadoAnalysis(
    T_WW=T_WW,
    T_CW=T_CW,
    p_gross=P_GROSS,
    cost_level=COST_LEVEL
)

print("Running Tornado analysis...")
tornado_results = tornado.run(output='lcoe', use_bounds=True, show_progress=True)
print("Done!")

In [ ]:
# Display results
print(f"\nTornado Analysis Results")
print(f"=" * 60)
print(f"Baseline LCOE: {tornado_results.baseline:.2f} ct/kWh")
print(f"\n{'Rank':<6} {'Parameter':<40} {'Swing':>10}")
print(f"-" * 60)

for i, (name, swing) in enumerate(tornado_results.get_ranking(), 1):
    direction = "↑" if swing > 0 else "↓"
    print(f"{i:<6} {name:<40} {swing:>+9.2f} {direction}")

In [ ]:
# Visualize tornado diagram
fig, ax = plt.subplots(figsize=(12, 8))
plot_tornado(tornado_results, ax=ax, top_n=10)
plt.title('Parameter Sensitivity - Tornado Diagram', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('tornado_diagram.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. Sobol Sensitivity Analysis

Global sensitivity analysis using variance-based methods.

**Note:** Requires SALib package (`pip install SALib`)

In [ ]:
# Check if SALib is available
try:
    from otex.analysis import SobolAnalysis, plot_sobol_indices
    SALIB_AVAILABLE = True
    print("SALib is available - Sobol analysis enabled")
except ImportError:
    SALIB_AVAILABLE = False
    print("SALib not installed. Install with: pip install SALib")
    print("Skipping Sobol analysis...")

In [ ]:
if SALIB_AVAILABLE:
    # Run Sobol analysis
    sobol = SobolAnalysis(
        T_WW=T_WW,
        T_CW=T_CW,
        n_samples=256,  # Base samples (total = n * (2d + 2))
        calc_second_order=False,
        p_gross=P_GROSS,
        cost_level=COST_LEVEL
    )
    
    print("Running Sobol analysis (this may take a few minutes)...")
    sobol_results = sobol.run(output='lcoe', show_progress=True)
    print("Done!")
else:
    sobol_results = None

In [ ]:
if sobol_results is not None:
    # Display Sobol indices
    print(f"\nSobol Sensitivity Indices")
    print(f"=" * 60)
    print(f"{'Parameter':<40} {'S1':>8} {'ST':>8}")
    print(f"-" * 60)
    
    for name, st in sobol_results.get_ranking('ST'):
        idx = sobol_results.parameter_names.index(name)
        s1 = sobol_results.S1[idx]
        print(f"{name:<40} {s1:>8.3f} {st:>8.3f}")
    
    print(f"\nSum of S1: {np.sum(sobol_results.S1):.3f}")
    print(f"Sum of ST: {np.sum(sobol_results.ST):.3f}")

In [ ]:
if sobol_results is not None:
    # Visualize Sobol indices
    fig, ax = plt.subplots(figsize=(10, 8))
    plot_sobol_indices(sobol_results, ax=ax, top_n=10)
    plt.title('Sobol Sensitivity Indices', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig('sobol_indices.png', dpi=150, bbox_inches='tight')
    plt.show()

## 5. Custom Parameters

Define your own uncertain parameters for analysis.

In [ ]:
# Define custom uncertain parameters
custom_params = [
    # Economic parameters
    UncertainParameter(
        name='discount_rate',
        nominal=0.08,
        distribution='uniform',
        bounds=(0.05, 0.12),
        category='economic'
    ),
    UncertainParameter(
        name='capex_structure_factor',
        nominal=1.0,
        distribution='triangular',
        bounds=(0.8, 1.5),
        category='economic'
    ),
    
    # Technical parameters
    UncertainParameter(
        name='turbine_isentropic_efficiency',
        nominal=0.85,
        distribution='normal',
        bounds=(0.85, 0.02),  # mean, std
        category='efficiency'
    ),
    UncertainParameter(
        name='U_evap',
        nominal=4.5,
        distribution='uniform',
        bounds=(4.0, 5.0),
        category='thermodynamic'
    ),
]

print(f"Custom Parameters ({len(custom_params)} total)")
for p in custom_params:
    print(f"  {p.name}: {p.nominal} ({p.distribution})")

In [ ]:
# Run Monte Carlo with custom parameters
custom_config = UncertaintyConfig(
    parameters=custom_params,
    n_samples=300,
    seed=42,
    parallel=False
)

mc_custom = MonteCarloAnalysis(
    T_WW=T_WW,
    T_CW=T_CW,
    config=custom_config,
    p_gross=P_GROSS,
    cost_level=COST_LEVEL
)

print("Running Monte Carlo with custom parameters...")
custom_results = mc_custom.run(show_progress=True)

# Compare statistics
custom_stats = custom_results.compute_statistics()
print(f"\nLCOE with Custom Parameters")
print(f"Mean: {custom_stats['lcoe']['lcoe_mean']:.2f} ct/kWh")
print(f"Std:  {custom_stats['lcoe']['lcoe_std']:.2f} ct/kWh")

## 6. Comparative Analysis

Compare uncertainty across different scenarios.

In [ ]:
# Compare different temperature conditions
scenarios = [
    {'name': 'Caribbean (High ΔT)', 'T_WW': 29.0, 'T_CW': 4.5},
    {'name': 'Pacific (Medium ΔT)', 'T_WW': 27.0, 'T_CW': 5.5},
    {'name': 'Marginal (Low ΔT)', 'T_WW': 25.0, 'T_CW': 7.0},
]

scenario_results = []

for scenario in scenarios:
    print(f"\nAnalyzing: {scenario['name']}...")
    
    config = UncertaintyConfig(n_samples=200, seed=42, parallel=False)
    mc = MonteCarloAnalysis(
        T_WW=scenario['T_WW'],
        T_CW=scenario['T_CW'],
        config=config,
        p_gross=P_GROSS,
        cost_level=COST_LEVEL
    )
    
    results = mc.run(show_progress=False)
    stats = results.compute_statistics()
    
    scenario_results.append({
        'name': scenario['name'],
        'delta_T': scenario['T_WW'] - scenario['T_CW'],
        'results': results,
        'mean': stats['lcoe']['lcoe_mean'],
        'std': stats['lcoe']['lcoe_std'],
        'p5': stats['lcoe']['lcoe_p5'],
        'p95': stats['lcoe']['lcoe_p95']
    })

print("\nDone!")

In [ ]:
# Display comparison
print(f"\nScenario Comparison")
print(f"=" * 70)
print(f"{'Scenario':<25} {'ΔT':>6} {'Mean':>10} {'Std':>8} {'90% CI'}")
print(f"-" * 70)

for s in scenario_results:
    ci = f"[{s['p5']:.1f}, {s['p95']:.1f}]"
    print(f"{s['name']:<25} {s['delta_T']:>5.1f}°C {s['mean']:>9.2f} {s['std']:>7.2f} {ci}")

In [ ]:
# Visualize comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Box plot comparison
data = [s['results'].lcoe[~np.isnan(s['results'].lcoe)] for s in scenario_results]
labels = [s['name'] for s in scenario_results]

bp = axes[0].boxplot(data, labels=labels, patch_artist=True)
colors = ['#3498db', '#2ecc71', '#e74c3c']
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

axes[0].set_ylabel('LCOE (ct/kWh)', fontsize=11)
axes[0].set_title('LCOE Distribution by Scenario', fontsize=12, fontweight='bold')
axes[0].tick_params(axis='x', rotation=15)

# Mean with error bars
x = range(len(scenario_results))
means = [s['mean'] for s in scenario_results]
stds = [s['std'] for s in scenario_results]

axes[1].bar(x, means, yerr=stds, capsize=5, color=colors, alpha=0.7, edgecolor='black')
axes[1].set_xticks(x)
axes[1].set_xticklabels(labels, rotation=15)
axes[1].set_ylabel('LCOE (ct/kWh)', fontsize=11)
axes[1].set_title('Mean LCOE ± Std Dev', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig('scenario_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

## 7. Summary Figure

Create a comprehensive summary of all analyses.

In [ ]:
# Create summary figure
fig = create_summary_figure(
    mc_results=mc_results,
    tornado_results=tornado_results,
    sobol_results=sobol_results,
    output='lcoe'
)

fig.suptitle('OTEX Uncertainty Analysis Summary', fontsize=14, fontweight='bold', y=1.02)
plt.savefig('uncertainty_summary.png', dpi=150, bbox_inches='tight')
plt.show()

## 8. Export Results

Save results for further analysis.

In [ ]:
# Export Monte Carlo samples
df_mc = pd.DataFrame(mc_results.samples, columns=mc_results.parameter_names)
df_mc['lcoe'] = mc_results.lcoe
df_mc['net_power'] = mc_results.net_power
df_mc['capex'] = mc_results.capex

df_mc.to_csv('monte_carlo_samples.csv', index=False)
print(f"Saved Monte Carlo samples to 'monte_carlo_samples.csv'")
print(f"Shape: {df_mc.shape}")
df_mc.head()

In [ ]:
# Export tornado results
df_tornado = pd.DataFrame({
    'parameter': tornado_results.parameter_names,
    'low_value': tornado_results.low_values,
    'high_value': tornado_results.high_values,
    'swing': tornado_results.swings
})
df_tornado = df_tornado.sort_values('swing', key=abs, ascending=False)
df_tornado.to_csv('tornado_results.csv', index=False)
print(f"Saved Tornado results to 'tornado_results.csv'")
df_tornado

## Summary

This notebook demonstrated:

1. **Monte Carlo Analysis**: Full uncertainty propagation with LHS sampling
2. **Tornado Diagram**: Quick identification of influential parameters
3. **Sobol Analysis**: Rigorous global sensitivity indices
4. **Custom Parameters**: Define your own uncertainty distributions
5. **Comparative Analysis**: Compare scenarios with different conditions

### Key Findings

- **Discount rate** is typically the most influential parameter
- **Structure costs** and **turbine efficiency** are also significant
- LCOE uncertainty is typically 15-25% (CV)
- Higher temperature difference → Lower LCOE and less uncertainty

### Recommendations

- Use at least 1000 samples for production Monte Carlo
- Focus uncertainty reduction efforts on high-sensitivity parameters
- Report 90% confidence intervals, not just mean values